# IMDB: MLM Insertion

In [ ]:
import json
import heapq
import os
import random
from typing import List

%pip install -U datasets
%pip install transformers
import torch
import matplotlib.pyplot as plt
import numpy as np
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import DataLoader
from transformers import AutoModelForMaskedLM, AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments


plt.style.use('fivethirtyeight')

ROOT_DIR = "drive/My Drive/Colab Notebooks/nlp/results/twitter_mlm_insertion"
if not os.path.exists(ROOT_DIR):
    os.mkdir(ROOT_DIR)

Requirement already up-to-date: datasets in /usr/local/lib/python3.6/dist-packages (1.1.2)


## Defining augmentation function

In [ ]:
class MLMInsertionAugmenter:
    def __init__(self, model, tokenizer, p: float, min_mask: int = 1, topk: int = 5, uniform: bool = False, device=None):
        self.device = device or torch.device('cpu')
        self.model = model.eval().to(self.device)
        self.tokenizer = tokenizer
        self.vocab_words = list(tokenizer.get_vocab().keys())
        self.mask_token = tokenizer.mask_token
        self.mask_token_id = tokenizer.mask_token_id
        self.topk = topk
        self.min_mask = min_mask
        self.uniform = uniform
        self.p = p
        
    def __call__(self, text: str):
        if self.p == 0:
            return text
        words = np.array(text.split(), dtype='object')
        n_mask = max(self.min_mask, int(len(words) * self.p))
        masked_indices = np.sort(np.random.choice(len(words) + 1, size=n_mask))

        masked_words = np.insert(words, masked_indices, self.mask_token)
        masked_text = " ".join(masked_words)
        
        tokenizer_output = self.tokenizer([masked_text])
        input_ids = torch.tensor(tokenizer_output['input_ids']).to(self.device)
        attention_mask = torch.tensor(tokenizer_output['attention_mask']).to(self.device)
        with torch.no_grad():
            output = self.model(input_ids)
            predicted_logits = output.logits[input_ids == self.mask_token_id]
            predicted_probas = predicted_logits.softmax(1)
            
        predicted_words = [self.sample_word(probas).strip() for probas in predicted_probas]
        
        new_words = np.insert(words, masked_indices, predicted_words)
        new_text = " ".join(new_words)
        return new_text
    
    
    def sample_word(self, predicted_probas):
        if hasattr(predicted_probas, 'tolist'):
            predicted_probas = predicted_probas.tolist()
        most_probable = heapq.nlargest(self.topk, zip(self.vocab_words, predicted_probas), key=lambda t: t[1])
        words, probas = zip(*most_probable)
        word = random.choice(words) if self.uniform else random.choices(words, weights=probas)[0]
        return self.tokenizer.convert_tokens_to_string(word).strip()

In [ ]:
class DatasetWithAugmentation(torch.utils.data.Dataset):
    def __init__(self, dataset, augmenter, augmentation_prob: float = 0.9):
        self.dataset = dataset
        self.augmenter = augmenter
        self.augmentation_prob = augmentation_prob

    def __getitem__(self, i):
        try:
            item = self.dataset[i]
            if random.random() < self.augmentation_prob:
                item['text'] = self.augmenter(item['text'])
            return item
        except:
            print(item)
            raise Exception(f"Something went wrong when augmenting item number {i}")

    def __len__(self):
        return len(self.dataset)


def get_datasets(dataset_name, augmenter, train_size, val_size=5_000, test_size=None, augmentation_prob=0.5, random_seed: int = 42):
    """Returns a tuple of train, validation and test datasets of sizes determined by arguments."""
    dataset = load_dataset(dataset_name, split="train")
    # We want test and validation data to be the same for every experiment
    test_split = dataset.train_test_split(test_size=test_size, seed=random_seed)
    test_dataset = test_split["test"]
    train_val_split = test_split["train"].train_test_split(test_size=val_size, seed=random_seed)
    # Validation and test sets
    train_dataset = train_val_split["train"].train_test_split(train_size=train_size, seed=random_seed)["train"]
    train_dataset = DatasetWithAugmentation(train_dataset, augmenter, augmentation_prob=augmentation_prob)
    val_dataset = train_val_split["test"]
    return train_dataset, val_dataset, test_dataset


class DataCollator:
    def __init__(self, tokenizer, label_colname='label'):
        self.tokenizer = tokenizer
        self.label_colname = label_colname
        
    def __call__(self, examples: List[dict]):
        labels = [0 if example['sentiment'] == 0 else 1 for example in examples]
        texts = [example['text'] for example in examples]
        tokenizer_output = self.tokenizer(texts, truncation=True, padding=True)
        return {
            'labels': torch.tensor(labels), 
            'input_ids': torch.tensor(tokenizer_output['input_ids']), 
            'attention_mask': torch.tensor(tokenizer_output['attention_mask'])
            }


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
AUGMENTATION_PROB = 0.7

tokenizer = AutoTokenizer.from_pretrained('roberta-base', use_fast=False)  # we cannot use Fast tokenizer for MLMInsertionAugmenter
data_collator = DataCollator(tokenizer, 'sentiment')

device = torch.device('cuda')
mlm_model = AutoModelForMaskedLM.from_pretrained('roberta-base', return_dict=True).eval()

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
augmenter = MLMInsertionAugmenter(mlm_model, tokenizer, 0.2, min_mask=1, topk=10, uniform=False, device=torch.device('cuda'))

In [ ]:
augmenter('I love you ')

'I love you .'

In [ ]:
train_size = 100

FRACTIONS = [0.0, 0.01, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6]
accuracies = list()

for fraction in FRACTIONS:

    model = AutoModelForSequenceClassification.from_pretrained('roberta-base', return_dict=True)
    augmenter = MLMInsertionAugmenter(mlm_model, tokenizer, fraction, min_mask=1, topk=10, uniform=False, device=device)

    train_dataset, val_dataset, test_dataset = get_datasets("sentiment140", augmenter, train_size, val_size=1_000, test_size=20_000, augmentation_prob=AUGMENTATION_PROB)
    print(f"Train size: {len(train_dataset)}, Validation size: {len(val_dataset)}, Test size: {len(test_dataset)}")
    print(f"Augmentation fraction: {fraction}")
    print(train_dataset[0])
    print(val_dataset[0])
    print(test_dataset[0])
    output_dir = os.path.join(ROOT_DIR, f"train_size_{train_size}_p_{fraction}")

    num_train_epochs = 9

    # https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments
    training_args = TrainingArguments(
        learning_rate=3e-5,
        weight_decay=0.01,
        output_dir=output_dir,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=2,
        warmup_steps=0,  # don't have any intuition for the right value here
        logging_dir=output_dir,
        logging_steps=5,
        load_best_model_at_end=False,
        evaluation_strategy='epoch',
        remove_unused_columns=False,
        no_cuda=False,
        metric_for_best_model="eval_accuracy"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        compute_metrics=compute_metrics
        
    )

    trainer.train()

    test_result = trainer.evaluate(test_dataset)

    print(test_result)
    accuracies.append(test_result['eval_accuracy'])


    with open(os.path.join(output_dir, 'test_result.json'), 'w') as f:
        json.dump(test_result, f, indent=4)

    print()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Train size: 100, Validation size: 1000, Test size: 20000
Augmentation fraction: 0.0
{'date': 'Mon May 04 06:28:55 PDT 2009', 'query': 'NO_QUERY', 'sentiment': 4, 'text': "2nd result of the day: found my timeout book of bars whiiich means i've now recovered everything I thought i'd lost. Small victories = ", 'user': 'Bandrew'}
{'date': 'Wed Jun 24 23:21:53 PDT 2009', 'query': 'NO_QUERY', 'sentiment': 0, 'text': "yea!!!! I think I have someone to watch the dogs while we're away. just have to kennel for 2 days instead of 11. no one wanted Zhu ", 'user': 'lindentreephoto'}
{'date': 'Sun Jun 07 07:43:33 PDT 2009', 'query': 'NO_QUERY', 'sentiment': 0, 'text': 'why am i awake so early?  damn projects. super nervous for the science one. mines gunna be so lame i dont wanna fail ', 'user': '_stacey_rae'}


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,0.688572,0.691712,0.506000,0.671979,0.506000,1.000000
1,0.762655,0.690137,0.507000,0.672425,0.506507,1.000000
2,0.738753,0.684074,0.515000,0.674715,0.510660,0.994071
3,0.716569,0.658528,0.569000,0.690151,0.542373,0.948617
4,0.505957,0.600710,0.683000,0.717720,0.653160,0.796443
5,0.450586,0.561520,0.707000,0.703741,0.720497,0.687747
6,0.300618,0.593582,0.721000,0.749776,0.686371,0.826087
7,0.224942,0.555019,0.739000,0.741328,0.743539,0.739130
8,0.171442,0.554884,0.748000,0.750495,0.751984,0.749012


{'eval_loss': 0.5538463592529297, 'eval_accuracy': 0.7421, 'eval_f1': 0.7435616983195785, 'eval_precision': 0.7390058306156735, 'eval_recall': 0.7481740870435217, 'epoch': 8.923076923076923, 'total_flos': 26391296009760}



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

...

{'eval_loss': 0.5703228712081909, 'eval_accuracy': 0.7193, 'eval_f1': 0.7230936174410575, 'eval_precision': 0.7131043875863411, 'eval_recall': 0.7333666833416709, 'epoch': 8.923076923076923, 'total_flos': 31602544893120}



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Train size: 100, Validation size: 1000, Test size: 20000
Augmentation fraction: 0.6
{'date': 'Mon May 04 06:28:55 PDT 2009', 'query': 'NO_QUERY', 'sentiment': 4, 'text': "2nd result of the day: found my timeout book of bars whiiich means i've now recovered everything I thought i'd lost. Small victories = ", 'user': 'Bandrew'}
{'date': 'Wed Jun 24 23:21:53 PDT 2009', 'query': 'NO_QUERY', 'sentiment': 0, 'text': "yea!!!! I think I have someone to watch the dogs while we're away. just have to kennel for 2 days instead of 11. no one wanted Zhu ", 'user': 'lindentreephoto'}
{'date': 'Sun Jun 07 07:43:33 PDT 2009', 'query': 'NO_QUERY', 'sentiment': 0, 'text': 'why am i awake so early?  damn projects. super nervous for the science one. mines gunna be so lame i dont wanna fail ', 'user': '_stacey_rae'}


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,0.699085,0.697672,0.506000,0.671979,0.506000,1.000000
1,0.772871,0.693380,0.506000,0.671979,0.506000,1.000000
2,0.793959,0.689596,0.507000,0.672425,0.506507,1.000000
3,0.738652,0.685885,0.539000,0.682725,0.523759,0.980237
4,0.666360,0.678137,0.573000,0.692141,0.544835,0.948617
5,0.710661,0.659776,0.658000,0.704663,0.625767,0.806324
6,0.666724,0.637116,0.655000,0.716049,0.613540,0.859684
7,0.612968,0.611571,0.664000,0.716216,0.625369,0.837945
8,0.573511,0.597496,0.681000,0.720421,0.647244,0.812253


{'eval_loss': 0.5979194045066833, 'eval_accuracy': 0.69195, 'eval_f1': 0.7294840834248079, 'eval_precision': 0.65, 'eval_recall': 0.8311155577788895, 'epoch': 8.923076923076923, 'total_flos': 32841039174240}



In [ ]:
plt.plot(FRACTIONS, accuracies)
plt.scatter(FRACTIONS, accuracies)
plt.title('Twitter')
plt.xlabel('Augmentation strength')
plt.ylabel('Accuracy')
plt.savefig(os.path.join(ROOT_DIR, 'twitter_1000_mlm_insertion_strength.eps'), format='eps', bbox_inches='tight')